In [6]:
import pandas as pd
import numpy as np
import networkx as nx
import utilities as ut

In [27]:
df  = pd.read_csv("GBPT/edges.csv")
#Extracting London PT
node_df = pd.read_csv("GBPT/nodes.csv")
#Creating the london df
# london_nodes_df = node_df.loc[node_df["zone"]==490]
# london_nodes = list(london_nodes_df["node"])
# ori_node = [];des_node=[];ori_layer=[];des_layer =[]
# for index,rows in df.iterrows():
#     if rows["ori_node"] in london_nodes:
#         ori_node.append(rows["ori_node"])
#         des_node.append(rows["des_node"])
#         ori_layer.append(rows["ori_layer"])
#         des_layer.append(rows["des_layer"])
        

# london_df = pd.DataFrame({"ori_node":ori_node,
#                           "des_node":des_node,
#                           "ori_layer":ori_layer,
#                           "des_layer":des_layer
#                           })
# london_df.to_csv("GBPT/processed/london_df.csv")

rail_df = df.loc[df["ori_layer"] == 2]
rail_df = rail_df.loc[rail_df["des_layer"] ==2]
source = rail_df["ori_node"]
target = rail_df["des_node"]
graph_df = pd.DataFrame(data={"source":source,"target":target})
G = ut.df_to_network(graph_df)
G = G.remove_edges_from(nx.selfloop_edges(G))
graph_df = ut.network_to_df(G)

AttributeError: 'NoneType' object has no attribute 'edges'

,source,target
367,1,19713
368,1,34938
369,22,36247
370,22,41454
371,58,351
...,...,...
8597,261754,36840
8598,261754,57479
8599,261754,107204
8600,261754,169164


In [7]:
#Beijing buses process

df = pd.read_csv("bus/raw/beijing.csv")
stops = set(df["站点名称"])
lines = set(df["线路名称"])
lines_id = np.linspace(1,len(lines),len(lines))
lines_id = [int(item) for item in lines_id]
stops_id = np.linspace(1,len(stops),len(stops))
stops_id = [int(item) for item in stops_id]

line_dict = dict(zip(lines_id,lines))
stop_dict = dict(zip(stops_id,stops))
stops_id_conversion =[]
for stop in df["站点名称"]:
    stops_id_conversion.append(list(stop_dict.values()).index(stop))
stops_id_conversion = [stop+1 for stop in stops_id_conversion]
df["Stop_ID"] = stops_id_conversion



lines_list_seperate = []
for item in list(lines):
    temp_list = df.loc[df["线路名称"]==item]
    lines_list_seperate.append(temp_list)
    
G = nx.Graph()
G.add_nodes_from(stops_id)

for lines in lines_list_seperate:
    for i in range(len(lines)):
        for first, second in zip(lines["Stop_ID"], lines["Stop_ID"][1:]):
            G.add_edge(first,second)

#Remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))

edges = list(G.edges())
source = [item[0] for item in edges]
target = [item[1] for item in edges]
data = {"source" : source,"target" : target}
processed_beijing_bus = pd.DataFrame(data = data)
processed_beijing_bus.to_csv("bus/processed/beijing.csv",index = False)

Gcc = ut.gcc(G)
edges = list(Gcc.edges())
source = [item[0] for item in edges]
target = [item[1] for item in edges]
data = {"source" : source,"target" : target}
processed_beijing_bus_simple = pd.DataFrame(data = data)
processed_beijing_bus_simple.to_csv("bus/simple/beijing_bus_simple.csv",index= False)

In [9]:
df

,ori_node,des_node,ori_layer,des_layer,minutes,km
0,325,31793,1,1,7,0.707
1,338,36564,1,1,7,0.920
2,348,46689,1,1,5,0.439
3,601,44530,1,1,495,164.555
4,601,44678,1,1,465,212.353
...,...,...,...,...,...,...
475497,261577,261577,5,4,1,0.000
475498,262053,262053,4,5,1,0.000
475499,262053,262053,5,4,1,0.000
475500,262062,262062,4,5,1,0.000
